In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import time

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

! pip install visualkeras
import visualkeras


import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
print(tf.test.is_gpu_available())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
#Pre-process image size
IMAGE_RES = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE

#Training parameters
LEARNING_RATE = 0.001
BATCH_SIZE = 64
EPOCHS = 300

In [ ]:
meta_df =  pd.read_csv('../input/gtsrb-german-traffic-sign/Meta.csv')
n_classes = meta_df["ClassId"].nunique()
n_classes

In [ ]:
data_df = pd.read_csv('../input/gtsrb-german-traffic-sign/Train.csv',usecols=['ClassId', 'Path'])
data_df.head()

In [ ]:
test_df = pd.read_csv('../input/gtsrb-german-traffic-sign/Test.csv',usecols=['ClassId', 'Path'])
test_df.head()

In [ ]:
dist = data_df['ClassId'].value_counts()

plt.figure(figsize=(21, 8))
plt.bar(dist.index, dist.values)
plt.xlabel('Classes')
plt.ylabel('Count of classes')
plt.xticks(dist.index, rotation='vertical')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
X = "../input/gtsrb-german-traffic-sign/" + data_df['Path'].values
y = data_df['ClassId'].values
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state = 42)

X_test = "../input/gtsrb-german-traffic-sign/" + test_df['Path'].values
y_test = test_df['ClassId'].values

In [ ]:
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMAGE_RES, IMAGE_RES])
    image /= 255.0  # normalize to [0,1] range

    return image

def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)

def load_and_preprocess_from_path_label(path, label):
    return load_and_preprocess_image(path), label

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_batches = train_ds.map(load_and_preprocess_from_path_label).batch(BATCH_SIZE).prefetch(1)

valid_ds = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))
validation_batches = valid_ds.map(load_and_preprocess_from_path_label).batch(BATCH_SIZE).prefetch(1)

test_data = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_batches = test_data.map(load_and_preprocess_from_path_label).batch(BATCH_SIZE).prefetch(1)

In [ ]:
# Create new LeNet-5 model
model = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(IMAGE_RES, IMAGE_RES, 3)),
    layers.experimental.preprocessing.RandomRotation(0.2),
    layers.Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='relu',padding="valid"),
    layers.MaxPooling2D(pool_size=(2, 2),strides=(1, 1), padding='valid'),
    layers.Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='relu',padding="valid"),
    layers.MaxPooling2D(pool_size=(2, 2),strides=(1, 1), padding='valid'),
    layers.Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='relu',padding="valid"),
    layers.MaxPooling2D(pool_size=(2, 2),strides=(1, 1), padding='valid'),
    layers.Conv2D(64, kernel_size=(5, 5), strides=(1, 1), activation='relu',padding="valid"),
    layers.MaxPooling2D(pool_size=(2, 2),strides=(1, 1), padding='valid'),
    layers.Conv2D(128, kernel_size=(5, 5), strides=(1, 1), activation='relu',padding="valid"),
    layers.MaxPooling2D(pool_size=(2, 2),strides=(1, 1), padding='valid'),   
    layers.Conv2D(256, kernel_size=(5, 5), strides=(1, 1), activation='relu',padding="valid"),
    layers.MaxPooling2D(pool_size=(2, 2),strides=(1, 1), padding='valid'), 
    layers.Flatten(),
    layers.Dense(120, activation='relu',kernel_regularizer='l2'),
    layers.Dense(84, activation='relu',kernel_regularizer='l2'),
    layers.Dense(n_classes)
])

model.summary()

In [ ]:
visualkeras.layered_view(model, type_ignore=[layers.Flatten], legend=True)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=20)

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

history = model.fit(train_batches, epochs=EPOCHS, validation_data=validation_batches, callbacks=[callback])

In [ ]:
# Evaluate the model on the validation set
print("Evaluate on validation data")
results = model.evaluate(validation_batches)
print("validation loss, validation acc:", results)

In [ ]:
# Evaluate the model on the test set
print("Evaluate on test data")
results = model.evaluate(test_batches)
print("Test loss, test acc:", results)

In [ ]:
#Saved Model
# t = time.time()

# export_path_sm = "./save_model/LeNet/{}".format(int(t))
# print(export_path_sm)

# tf.saved_model.save(model, export_path_sm)